In [6]:
# MasterDatabaseManagement/Changes/cleanup_mx_contracts.py
"""
Script de limpieza para contratos MX>=0108 que tienen problemas de sincronización.
Borra en orden correcto respetando foreign keys: CA → CTI → FPI
"""

from sqlalchemy import text
from core.db import get_engine
from core.libs import pd
from typing import List, Set


def get_mx_contracts_to_clean(start_seq: int = 108) -> List[str]:
    """
    Obtiene todos los contract_codes MX >= start_seq desde CTI
    """
    engine = get_engine()
    sql = text("""
        SELECT contract_code
        FROM masterdatabase.contract_tree_information
        WHERE contract_code LIKE 'MX%'
          AND CAST(SUBSTRING(contract_code FROM 3) AS INTEGER) >= :start_seq
        ORDER BY contract_code
    """)

    with engine.begin() as conn:
        result = conn.execute(sql, {"start_seq": start_seq})
        return [row[0] for row in result]


def get_affected_farmer_numbers(contract_codes: List[str]) -> Set[str]:
    """
    Obtiene farmer_numbers que tienen SOLO estos contratos (para limpieza completa)
    """
    if not contract_codes:
        return set()

    engine = get_engine()

    # Buscar farmers que SOLO tienen estos contratos
    # Usamos array operators correctamente
    sql = text("""
        SELECT DISTINCT farmer_number
        FROM masterdatabase.farmer_personal_information
        WHERE farmer_number IS NOT NULL
          AND contract_codes IS NOT NULL
          AND array_length(contract_codes, 1) > 0
          AND contract_codes <@ CAST(:codes AS text[])  -- todos sus contratos están en el set
    """)

    with engine.begin() as conn:
        result = conn.execute(sql, {"codes": contract_codes})
        farmers_only_these = {row[0] for row in result}

    return farmers_only_these


def cleanup_contracts(contract_codes: List[str], dry_run: bool = True):
    """
    Limpia contratos en orden correcto: CA → CTI → FPI
    """
    if not contract_codes:
        print("⚠️  No hay contratos para limpiar")
        return

    engine = get_engine()

    print(f"\n{'='*60}")
    print(f"🧹 LIMPIEZA DE CONTRATOS MX (dry_run={dry_run})")
    print(f"{'='*60}")
    print(f"📋 Contratos a eliminar: {len(contract_codes)}")
    print(f"   Rango: {min(contract_codes)} - {max(contract_codes)}")

    # Obtener farmer_numbers afectados
    farmers_to_clean = get_affected_farmer_numbers(contract_codes)
    print(f"👥 Farmers que SOLO tienen estos contratos: {len(farmers_to_clean)}")

    if dry_run:
        print("\n🔍 MODO DRY RUN - Mostrando qué se eliminaría:\n")

        # Preview CA
        sql_ca = text("""
            SELECT ca.contract_code, COUNT(*) as records
            FROM masterdatabase.contract_allocation ca
            WHERE ca.contract_code = ANY(CAST(:codes AS text[]))
            GROUP BY ca.contract_code
            ORDER BY ca.contract_code
        """)
        with engine.begin() as conn:
            df_ca = pd.read_sql(sql_ca, conn, params={"codes": contract_codes})

        if not df_ca.empty:
            print("📊 Contract Allocation (registros por año):")
            print(df_ca.to_string(index=False))
            print(f"   Total registros CA: {df_ca['records'].sum()}")
        else:
            print("📊 Contract Allocation: ✅ Sin registros")

        # Preview CTI
        sql_cti = text("""
            SELECT contract_code, planting_year, trees_contract, status
            FROM masterdatabase.contract_tree_information
            WHERE contract_code = ANY(CAST(:codes AS text[]))
            ORDER BY contract_code
        """)
        with engine.begin() as conn:
            df_cti = pd.read_sql(sql_cti, conn, params={"codes": contract_codes})

        if not df_cti.empty:
            print(f"\n🌳 Contract Tree Information:")
            print(df_cti.to_string(index=False))
            print(f"   Total registros CTI: {len(df_cti)}")

        # Preview FPI
        if farmers_to_clean:
            sql_fpi = text("""
                SELECT farmer_number, representative, contract_codes
                FROM masterdatabase.farmer_personal_information
                WHERE farmer_number = ANY(CAST(:farmers AS text[]))
                ORDER BY farmer_number
            """)
            with engine.begin() as conn:
                df_fpi = pd.read_sql(sql_fpi, conn, params={"farmers": list(farmers_to_clean)})

            if not df_fpi.empty:
                print(f"\n👤 Farmer Personal Information (eliminación completa):")
                print(df_fpi.to_string(index=False))

        # Farmers que se actualizarán (remover códigos)
        sql_fpi_update = text("""
            SELECT farmer_number, representative, contract_codes
            FROM masterdatabase.farmer_personal_information
            WHERE EXISTS (
                SELECT 1 FROM unnest(contract_codes) AS cc
                WHERE cc = ANY(CAST(:codes AS text[]))
            )
            AND NOT (contract_codes <@ CAST(:codes AS text[]))
            ORDER BY farmer_number
        """)
        with engine.begin() as conn:
            df_fpi_upd = pd.read_sql(sql_fpi_update, conn, params={"codes": contract_codes})

        if not df_fpi_upd.empty:
            print(f"\n👤 Farmer Personal Information (actualización - remover códigos):")
            print(df_fpi_upd.to_string(index=False))

        print("\n" + "="*60)
        print("✅ DRY RUN completado. Ejecuta con --execute para aplicar cambios.")
        print("="*60)
        return

    # EJECUCIÓN REAL
    print("\n⚠️  MODO EJECUCIÓN - Aplicando cambios...\n")

    with engine.begin() as conn:
        # PASO 1: Eliminar de Contract Allocation
        sql_delete_ca = text("""
            DELETE FROM masterdatabase.contract_allocation
            WHERE contract_code = ANY(CAST(:codes AS text[]))
        """)
        result_ca = conn.execute(sql_delete_ca, {"codes": contract_codes})
        print(f"✅ Contract Allocation: {result_ca.rowcount} registros eliminados")

        # PASO 2: Eliminar de Contract Tree Information
        sql_delete_cti = text("""
            DELETE FROM masterdatabase.contract_tree_information
            WHERE contract_code = ANY(CAST(:codes AS text[]))
        """)
        result_cti = conn.execute(sql_delete_cti, {"codes": contract_codes})
        print(f"✅ Contract Tree Information: {result_cti.rowcount} registros eliminados")

        # PASO 3: Actualizar FPI - remover códigos de arrays
        sql_update_fpi = text("""
            UPDATE masterdatabase.farmer_personal_information
            SET contract_codes = array_remove_all(contract_codes, :codes::text[])
            WHERE contract_codes && :codes::text[]
        """)
        # Remover múltiples valores del array
        for code in contract_codes:
            sql_remove_one = text("""
                UPDATE masterdatabase.farmer_personal_information
                SET contract_codes = array_remove(contract_codes, :code)
                WHERE :code = ANY(contract_codes)
                  AND array_length(contract_codes, 1) > 1
            """)
            conn.execute(sql_remove_one, {"code": code})

        print(f"✅ FPI Arrays actualizados: contratos removidos de arrays")

        # PASO 4: Eliminar farmers que quedaron sin contratos
        if farmers_to_clean:
            sql_delete_fpi = text("""
                DELETE FROM masterdatabase.farmer_personal_information
                WHERE farmer_number = ANY(CAST(:farmers AS text[]))
            """)
            result_fpi_del = conn.execute(sql_delete_fpi, {"farmers": list(farmers_to_clean)})
            print(f"✅ FPI eliminados: {result_fpi_del.rowcount} farmers")

    print("\n" + "="*60)
    print("✅ LIMPIEZA COMPLETADA")
    print("="*60)
    print("\n💡 Siguiente paso: Re-ejecutar new_contract_input_activation.py")


def main():
    import sys

    # Parsear argumentos
    dry_run = "--execute" not in sys.argv
    start_seq = 108

    # Permitir especificar secuencia inicial
    for arg in sys.argv:
        if arg.startswith("--start="):
            start_seq = int(arg.split("=")[1])

    print("🔍 Buscando contratos MX a limpiar...")
    contracts = get_mx_contracts_to_clean(start_seq)

    if not contracts:
        print(f"✅ No se encontraron contratos MX >= {start_seq:04d}")
        return

    cleanup_contracts(contracts, dry_run=dry_run)


if __name__ == "__main__":
    main()

🔍 Buscando contratos MX a limpiar...
💻 Conectado a la base de datos helloworldtree
💻 Conectado a la base de datos helloworldtree

🧹 LIMPIEZA DE CONTRATOS MX (dry_run=True)
📋 Contratos a eliminar: 21
   Rango: MX0108 - MX0128
💻 Conectado a la base de datos helloworldtree
👥 Farmers que SOLO tienen estos contratos: 17

🔍 MODO DRY RUN - Mostrando qué se eliminaría:

📊 Contract Allocation (registros por año):
contract_code  records
       MX0108        1
       MX0109        1
       MX0110        1
       MX0111        1
       MX0112        1
       MX0113        1
       MX0114        1
       MX0115        1
       MX0116        1
       MX0117        1
       MX0118        1
       MX0119        1
       MX0120        1
       MX0121        1
       MX0122        1
       MX0123        1
       MX0124        1
       MX0125        1
       MX0126        1
       MX0127        1
       MX0128        1
   Total registros CA: 21

🌳 Contract Tree Information:
contract_code  planting_year  

In [7]:
# === LIMPIEZA DE CONTRATOS MX0108-MX0128 ===
# Ejecutar este cell para limpiar los contratos problemáticos

from sqlalchemy import text
from core.db import get_engine
from core.libs import pd

# Contratos a eliminar (basado en el dry-run)
contracts_to_delete = [
    'MX0108', 'MX0109', 'MX0110', 'MX0111', 'MX0112', 'MX0113', 'MX0114',
    'MX0115', 'MX0116', 'MX0117', 'MX0118', 'MX0119', 'MX0120', 'MX0121',
    'MX0122', 'MX0123', 'MX0124', 'MX0125', 'MX0126', 'MX0127', 'MX0128'
]

# Farmers que serán eliminados completamente (solo tienen estos contratos)
farmers_to_delete = [
    '40099', '40100', '40101', '40102', '40103', '40104', '40105',
    '40106', '40107', '40108', '40109', '40110', '40111', '40112',
    '40113', '40114', '40115'
]

print("="*60)
print("🧹 LIMPIEZA DE CONTRATOS MX")
print("="*60)
print(f"📋 Contratos a eliminar: {len(contracts_to_delete)}")
print(f"👥 Farmers a eliminar: {len(farmers_to_delete)}")
print()

engine = get_engine()

# ========== PASO 1: Eliminar de Contract Allocation ==========
print("🗑️  PASO 1: Eliminando de Contract Allocation...")
with engine.begin() as conn:
    result = conn.execute(
        text("""
            DELETE FROM masterdatabase.contract_allocation
            WHERE contract_code = ANY(CAST(:codes AS text[]))
        """),
        {"codes": contracts_to_delete}
    )
    print(f"   ✅ {result.rowcount} registros eliminados de CA")

# ========== PASO 2: Eliminar de Contract Tree Information ==========
print("🗑️  PASO 2: Eliminando de Contract Tree Information...")
with engine.begin() as conn:
    result = conn.execute(
        text("""
            DELETE FROM masterdatabase.contract_tree_information
            WHERE contract_code = ANY(CAST(:codes AS text[]))
        """),
        {"codes": contracts_to_delete}
    )
    print(f"   ✅ {result.rowcount} registros eliminados de CTI")

# ========== PASO 3: Actualizar arrays en FPI (remover códigos) ==========
print("🗑️  PASO 3: Actualizando arrays en FPI...")
with engine.begin() as conn:
    # Remover códigos de farmers que tienen otros contratos
    for code in contracts_to_delete:
        conn.execute(
            text("""
                UPDATE masterdatabase.farmer_personal_information
                SET contract_codes = array_remove(contract_codes, :code)
                WHERE :code = ANY(contract_codes)
            """),
            {"code": code}
        )
    print(f"   ✅ Códigos removidos de arrays")

# ========== PASO 4: Eliminar farmers que quedaron sin contratos ==========
print("🗑️  PASO 4: Eliminando farmers sin contratos...")
with engine.begin() as conn:
    result = conn.execute(
        text("""
            DELETE FROM masterdatabase.farmer_personal_information
            WHERE farmer_number = ANY(CAST(:farmers AS text[]))
        """),
        {"farmers": farmers_to_delete}
    )
    print(f"   ✅ {result.rowcount} farmers eliminados de FPI")

# ========== VERIFICACIÓN ==========
print()
print("="*60)
print("🔍 VERIFICACIÓN POST-LIMPIEZA")
print("="*60)

with engine.begin() as conn:
    # Verificar CA
    ca_count = conn.execute(
        text("""
            SELECT COUNT(*)
            FROM masterdatabase.contract_allocation
            WHERE contract_code = ANY(CAST(:codes AS text[]))
        """),
        {"codes": contracts_to_delete}
    ).scalar()
    print(f"📊 CA: {ca_count} registros (debería ser 0)")

    # Verificar CTI
    cti_count = conn.execute(
        text("""
            SELECT COUNT(*)
            FROM masterdatabase.contract_tree_information
            WHERE contract_code = ANY(CAST(:codes AS text[]))
        """),
        {"codes": contracts_to_delete}
    ).scalar()
    print(f"🌳 CTI: {cti_count} registros (debería ser 0)")

    # Verificar FPI
    fpi_count = conn.execute(
        text("""
            SELECT COUNT(*)
            FROM masterdatabase.farmer_personal_information
            WHERE farmer_number = ANY(CAST(:farmers AS text[]))
        """),
        {"farmers": farmers_to_delete}
    ).scalar()
    print(f"👤 FPI: {fpi_count} farmers (debería ser 0)")

    # Verificar arrays actualizados
    arrays_ok = conn.execute(
        text("""
            SELECT farmer_number, contract_codes
            FROM masterdatabase.farmer_personal_information
            WHERE EXISTS (
                SELECT 1 FROM unnest(contract_codes) AS cc
                WHERE cc = ANY(CAST(:codes AS text[]))
            )
        """),
        {"codes": contracts_to_delete}
    ).fetchall()

    if arrays_ok:
        print(f"⚠️  Arrays: {len(arrays_ok)} farmers todavía tienen códigos problemáticos:")
        for row in arrays_ok:
            print(f"   - {row[0]}: {row[1]}")
    else:
        print(f"✅ Arrays: 0 farmers con códigos problemáticos")

print()
print("="*60)
print("✅ LIMPIEZA COMPLETADA")
print("="*60)
print()
print("💡 Siguiente paso: Re-ejecutar new_contract_input_activation.py")
print("   python MasterDatabaseManagement/Changes/new_contract_input_activation.py")

🧹 LIMPIEZA DE CONTRATOS MX
📋 Contratos a eliminar: 21
👥 Farmers a eliminar: 17

💻 Conectado a la base de datos helloworldtree
🗑️  PASO 1: Eliminando de Contract Allocation...
   ✅ 21 registros eliminados de CA
🗑️  PASO 2: Eliminando de Contract Tree Information...
   ✅ 21 registros eliminados de CTI
🗑️  PASO 3: Actualizando arrays en FPI...
   ✅ Códigos removidos de arrays
🗑️  PASO 4: Eliminando farmers sin contratos...
   ✅ 17 farmers eliminados de FPI

🔍 VERIFICACIÓN POST-LIMPIEZA
📊 CA: 0 registros (debería ser 0)
🌳 CTI: 0 registros (debería ser 0)
👤 FPI: 0 farmers (debería ser 0)
✅ Arrays: 0 farmers con códigos problemáticos

✅ LIMPIEZA COMPLETADA

💡 Siguiente paso: Re-ejecutar new_contract_input_activation.py
   python MasterDatabaseManagement/Changes/new_contract_input_activation.py


In [8]:
# Verificar qué farmers existen y cuáles tienen otros contratos
from sqlalchemy import text
from core.db import get_engine
from core.libs import pd

farmers_in_sheet = ['40067', '40053', '40050', '40072', '40074']

engine = get_engine()

print("="*80)
print("🔍 VERIFICACIÓN DE FARMERS EN EL SHEET")
print("="*80)

with engine.begin() as conn:
    result = conn.execute(
        text("""
            SELECT
                farmer_number,
                representative,
                contract_codes,
                array_length(contract_codes, 1) as num_contracts
            FROM masterdatabase.farmer_personal_information
            WHERE farmer_number = ANY(CAST(:farmers AS text[]))
            ORDER BY farmer_number
        """),
        {"farmers": farmers_in_sheet}
    )

    df = pd.DataFrame(result.fetchall(), columns=['farmer_number', 'representative', 'contract_codes', 'num_contracts'])

print(f"\n📊 Estado actual de farmers en FPI:")
print("="*80)

if df.empty:
    print("❌ NINGUNO de estos farmers existe en FPI")
    print("\n⚠️  PROBLEMA: Estás usando farmer_numbers que no existen.")
    print("\n🔧 SOLUCIONES:")
    print("   Opción A: Dejar la columna Farmer# VACÍA en el sheet")
    print("             → El script asignará automáticamente 40116, 40117, etc.")
    print("   Opción B: Usar farmer_numbers que SÍ existan en la BD")
else:
    print(f"✅ {len(df)} farmers SÍ EXISTEN en FPI:\n")
    print(df.to_string(index=False))
    print("\n" + "="*80)
    print("📋 ANÁLISIS POR FARMER:")
    print("="*80)
    for _, row in df.iterrows():
        fn = row['farmer_number']
        rep = row['representative']
        codes = row['contract_codes']
        num = row['num_contracts'] or 0

        print(f"\n👤 Farmer {fn}: {rep}")
        print(f"   Contratos actuales ({num}): {codes}")
        print(f"   ✅ Acción: CLONAR datos personales + APPEND nuevo código")

    # Verificar si alguno no existe
    existing = set(df['farmer_number'].tolist())
    missing = set(farmers_in_sheet) - existing

    if missing:
        print(f"\n" + "="*80)
        print(f"❌ FARMERS QUE NO EXISTEN: {len(missing)}")
        print("="*80)
        for fn in sorted(missing):
            print(f"  • {fn}: NO existe en FPI")
        print(f"\n⚠️  El script fallará al intentar clonar estos farmers.")
        print(f"\n🔧 SOLUCIÓN: En el sheet, BORRAR los farmer_numbers que no existen")
        print(f"             (dejar la celda vacía) para que se asignen automáticamente.")

print("\n" + "="*80)
print("✅ CONCLUSIÓN:")
print("="*80)

if not df.empty:
    existing = set(df['farmer_number'].tolist())
    missing = set(farmers_in_sheet) - existing

    if not missing:
        print("✅ TODOS los farmers existen → El script funcionará perfectamente")
        print("   Cada farmer será CLONADO y se le agregará el nuevo contract_code")
    else:
        print(f"⚠️  {len(missing)} farmers NO EXISTEN → El script crasheará")
        print(f"   Farmers problemáticos: {sorted(missing)}")
        print("\n📝 ACCIÓN REQUERIDA:")
        print("   1. Abre changelog.xlsx → NewContractInputLog")
        print("   2. BORRA los farmer_numbers que no existen:")
        for fn in sorted(missing):
            print(f"      - Fila con farmer {fn}: dejar celda Farmer# VACÍA")
        print("   3. El script asignará automáticamente números secuenciales")
else:
    print("❌ NINGÚN farmer existe → TODOS deben dejarse en blanco")
    print("   El script asignará: 40116, 40117, 40118, 40119, 40120")

# Verificar también el máximo actual para saber qué se asignará
with engine.begin() as conn:
    max_fn = conn.execute(
        text("""
            SELECT COALESCE(MAX(farmer_number::integer), 40000) AS maxnum
            FROM masterdatabase.farmer_personal_information
            WHERE farmer_number::integer BETWEEN 40000 AND 49999
        """)
    ).scalar()

print(f"\n📊 Máximo farmer_number MX actual: {max_fn}")
print(f"   → Próximos números automáticos: {max_fn+1}, {max_fn+2}, {max_fn+3}, ...")

💻 Conectado a la base de datos helloworldtree
🔍 VERIFICACIÓN DE FARMERS EN EL SHEET

📊 Estado actual de farmers en FPI:
✅ 5 farmers SÍ EXISTEN en FPI:

farmer_number             representative   contract_codes  num_contracts
        40050 Casto De la Cruz Francisco [MX0055, MX0103]              2
        40053                       None [MX0058, MX0101]              2
        40067                       None [MX0072, MX0100]              2
        40072            Rosario Sánchez         [MX0077]              1
        40074                       None         [MX0079]              1

📋 ANÁLISIS POR FARMER:

👤 Farmer 40050: Casto De la Cruz Francisco
   Contratos actuales (2): ['MX0055', 'MX0103']
   ✅ Acción: CLONAR datos personales + APPEND nuevo código

👤 Farmer 40053: None
   Contratos actuales (2): ['MX0058', 'MX0101']
   ✅ Acción: CLONAR datos personales + APPEND nuevo código

👤 Farmer 40067: None
   Contratos actuales (2): ['MX0072', 'MX0100']
   ✅ Acción: CLONAR datos personales